1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
    • Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
    • Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
    • Исключим стоп-слова с помощью stop_words='english'.
    • Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

In [4]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
with open('/content/drive/My Drive/clean.pkl', 'rb') as f:
    combine_df = pickle.load(f)

In [8]:
for col in ['tweet_stemmed', 'tweet_lemmatized']:
    combine_df[col] = combine_df[col].apply(lambda text_list: ' '.join(text_list))

combine_df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...",father dysfunct selfish drag kid dysfunct run,father dysfunctional selfish drag kid dysfunct...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit can use cause they do n...,"[thanks, for, lyft, credit, can, use, cause, t...","[thanks, lyft, credit, use, cause, offer, whee...",thank lyft credit use caus offer wheelchair va...,thanks lyft credit use cause offer wheelchair ...
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]",bihday majesti,bihday majesty
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in your,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time]",model love take time,model love take time
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]",factsguid societi motiv,factsguide society motivation


In [9]:
count_vectorizer = CountVectorizer(stop_words='english',
                                   max_df=0.9,
                                   max_features=1000)
stemmed = count_vectorizer.fit_transform(combine_df['tweet_stemmed'])
names_stemmed = count_vectorizer.get_feature_names()
stemmed_df = pd.DataFrame(stemmed.toarray(), columns=names_stemmed)
stemmed_df.head()

,abl,absolut,accept,account,act,action,activ,actor,actual,ad,adapt,add,adventur,affirm,afternoon,age,ago,agre,ahead,aist,album,aliv,allahsoil,allow,alon,alreadi,altwaystoh,alway,amaz,america,american,angel,anger,angri,anim,anniversari,announc,anoth,answer,anti,...,went,wet,whatev,white,wife,wild,win,wine,winner,wish,woh,woman,women,wonder,word,work,workout,world,worri,worst,wow,write,wrong,wtf,xx,xxx,ya,yay,ye,yeah,year,yesterday,yo,yoga,york,young,youtub,yoyou,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
lemmatized = count_vectorizer.fit_transform(combine_df['tweet_lemmatized'])
names_lemmatized = count_vectorizer.get_feature_names()
lemmatized_df = pd.DataFrame(lemmatized.toarray(), columns=names_lemmatized)
lemmatized_df.head()

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,affirmation,afternoon,age,ago,agree,ahead,aist,album,alive,allahsoil,altwaystoheal,amazing,america,american,angel,anger,angry,animal,anniversary,anti,anxiety,anymore,app,apple,arrived,ask,attack,august,available,award,...,white,wife,wild,win,wine,winner,wisdom,wish,wishing,woh,woman,wonder,wonderful,word,work,working,workout,world,worry,worst,wow,writing,wrong,wtf,xx,xxx,ya,yay,yeah,year,yes,yesterday,yo,yoga,york,young,youtube,yoyou,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
    • Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
    • Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
    • Исключим стоп-слова с помощью stop_words='english'.
    • Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().



In [11]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english',
                                   max_df=0.9,
                                   max_features=1000)
stemmed = tfidf_vectorizer.fit_transform(combine_df['tweet_stemmed'])
names_stemmed = tfidf_vectorizer.get_feature_names()
stemmed_df = pd.DataFrame(stemmed.toarray(), columns=names_stemmed)
stemmed_df.head()

,abl,absolut,accept,account,act,action,activ,actor,actual,ad,adapt,add,adventur,affirm,afternoon,age,ago,agre,ahead,aist,album,aliv,allahsoil,allow,alon,alreadi,altwaystoh,alway,amaz,america,american,angel,anger,angri,anim,anniversari,announc,anoth,answer,anti,...,went,wet,whatev,white,wife,wild,win,wine,winner,wish,woh,woman,women,wonder,word,work,workout,world,worri,worst,wow,write,wrong,wtf,xx,xxx,ya,yay,ye,yeah,year,yesterday,yo,yoga,york,young,youtub,yoyou,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
lemmatized = tfidf_vectorizer.fit_transform(combine_df['tweet_lemmatized'])
names_lemmatized = tfidf_vectorizer.get_feature_names()
lemmatized_df = pd.DataFrame(lemmatized.toarray(), columns=names_lemmatized)
lemmatized_df.head()

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,affirmation,afternoon,age,ago,agree,ahead,aist,album,alive,allahsoil,altwaystoheal,amazing,america,american,angel,anger,angry,animal,anniversary,anti,anxiety,anymore,app,apple,arrived,ask,attack,august,available,award,...,white,wife,wild,win,wine,winner,wisdom,wish,wishing,woh,woman,wonder,wonderful,word,work,working,workout,world,worry,worst,wow,writing,wrong,wtf,xx,xxx,ya,yay,yeah,year,yes,yesterday,yo,yoga,york,young,youtube,yoyou,yr,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


3. Натренируем gensim.models.Word2Vec модель на наших данных.
    • Тренировать будем на токенизированных твитах combine_df['tweet_token']
    • Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
    • Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.



In [13]:
import gensim.downloader as api
from gensim.models import Word2Vec, FastText

In [14]:
modelW2V = Word2Vec(sentences=combine_df['tweet_token'], size=200, window=5, min_count=2,
                    sg = 1, hs = 0, negative = 10, workers= 32, seed = 34)
modelW2V.train(combine_df['tweet_token'],
                total_examples=len(combine_df['tweet_token']),
                epochs=20)

(9114500, 11671780)

4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".


In [15]:
modelW2V.most_similar(positive='dinner')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bihdaydinner', 0.5550413131713867),
 ('bolognese', 0.5488468408584595),
 ('cookout', 0.544598400592804),
 ('spaghetti', 0.5241124629974365),
 ('tacotuesday', 0.5216798782348633),
 ('shawarma', 0.5207309722900391),
 ('waterloo', 0.5184285640716553),
 ('saturdate', 0.5163633823394775),
 ('lastnight', 0.5064916610717773),
 ('reggio', 0.5064623355865479)]

In [16]:
modelW2V.most_similar(positive=['trump'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('donald', 0.5452628135681152),
 ('dumptrump', 0.5346320271492004),
 ('crony', 0.5233725309371948),
 ('donaldtrump', 0.5139427185058594),
 ('impeachment', 0.5103570222854614),
 ('trumptrain', 0.502639889717102),
 ('fuhered', 0.5015271306037903),
 ('unfavorability', 0.5014222860336304),
 ('suppoer', 0.494256854057312),
 ('carl', 0.49309390783309937)]

5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".



In [17]:
modelW2V['food']

array([ 1.61672547e-01, -2.96499617e-02,  2.06366271e-01,  5.05826697e-02,
        6.89382255e-01,  1.35813892e-01,  8.93875420e-01, -2.21677795e-01,
        4.73715484e-01,  2.36534715e-01,  4.09286499e-01,  2.26819679e-01,
       -5.43554902e-01,  2.74128824e-01, -1.46567449e-01,  4.62489516e-01,
       -3.80793333e-01, -5.80302477e-01, -1.00766647e+00,  1.90588281e-01,
        1.05163157e-01,  8.35850358e-01,  2.47050449e-01,  1.03956729e-01,
        1.04134309e+00, -5.36411762e-01,  6.18198738e-02,  1.25045732e-01,
       -1.94720462e-01, -5.05456328e-02,  3.28217536e-01, -3.67014459e-03,
        4.41624820e-01, -3.12864423e-01, -2.48692587e-01,  5.91586344e-03,
       -1.83829144e-01,  1.10559225e-01, -4.06194568e-01,  2.06777781e-01,
       -8.81744206e-01, -3.45852107e-01,  1.25956833e-01,  7.58148953e-02,
        1.12061465e+00, -3.44535738e-01,  6.66878879e-01,  1.99427366e-01,
        6.61375344e-01,  9.94449761e-03, -4.84642655e-01, -1.59094751e-01,
       -5.00490606e-01, -

6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.
Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).

In [18]:
def text2vec(tweet,size=200):
    count = len(tweet)
    vec = np.zeros((1, size))
    for word in tweet:
        try:
            vec += modelW2V[word].reshape((1, size))
        except KeyError:
            continue
    if count == 0:
        return vec
    return vec / count

In [19]:
vec_df = combine_df['tweet_token'].apply(text2vec)

In [20]:
vec_df = np.concatenate(vec_df)
vec_df.shape

(49159, 200)

vec_df

2. Докрутить чат-бота (обучить лучший ft, w2v, взвесить idf). Модели сдампить и сложить в облако, ноутбук можно через гитхаб. Ответы: https://yadi.sk/d/SLgpFrmrBubeYg

In [27]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [29]:
# assert False

#Small preprocess of the answers

question = None
written = False

with open("/content/drive/My Drive/answers.txt", "w") as fout:
    with open("/content/drive/My Drive/Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [30]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [32]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("/content/drive/My Drive/Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

In [33]:
sentences = [i for i in sentences if len(i) > 2]

In [34]:
sentences[0]

['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести']

In [44]:
modelW2V = Word2Vec(sentences=sentences, size=30, window=3, min_count=2)
modelFT = FastText(sentences=sentences, size=30, min_count=2, window=4)

In [38]:
def tokenizer(text):
    return text
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenizer,
                                   lowercase=False, max_df=0.9)

In [39]:
tfidf_vectorizer.fit_transform(sentences)

<64662x165629 sparse matrix of type '<class 'numpy.float64'>'
	with 946117 stored elements in Compressed Sparse Row format>

In [40]:
tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_['вопрос']]

4.487608623568247

In [45]:


w2v_index = annoy.AnnoyIndex(30,'angular')
ft_index = annoy.AnnoyIndex(30,'angular')

index_map = {}
counter = 0

with open("/content/drive/My Drive/answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(30)
        vector_ft = np.zeros(30)
        for word in question:
            if word in modelW2V:
                vector_w2v += modelW2V[word]
                n_w2v += tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]] if word in tfidf_vectorizer.vocabulary_ else 1
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]] if word in tfidf_vectorizer.vocabulary_ else 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 50000:
            break

w2v_index.build(10)
ft_index.build(10)

True

In [46]:
w2v_index.get_nns_by_vector(np.zeros(30), 3)

[110, 793, 3024]

In [47]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(30)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]]
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [51]:
TEXT = "когда закончится карантин"

In [52]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Политик- это продукт соц. заказа. Вот например, Ленин, Сталин не наживались материально, у них были другие "приколы". Сейчас ВСЕ политиканы рвутся во власть за деньги и за блага хоть маленько оторваться от земли .. \n',
 'новый уровень-иногда. \n',
 'Еслиб не такие Халтурины, Россия былаб счастливой страной.. \n']

In [53]:
get_response(TEXT, ft_index, modelFT, index_map)

['Обратись на свою биржу труда, они не только направят, договорятся о индивидуальных занятиях, но еще и оплатят учебу и тебе степендию.. \n',
 'Постучу по дереву, останься пожалуйста невредимой после поездки. Чем больше сообщений из Турции о несчастных случаях, тем больше туда желающих. Там что, мёдом, что-ли намазанно? Лучше на Кипр (не в целях рекламмы).. \n',
 'со своим ангелом..Чтобы она для меня была лучиком надежды в этом темном царстве. \n']

результаты какие есть 